In [1]:
from pynq import Overlay
from pynq import Xlnk
import matplotlib.pyplot as plt
import numpy as np
import cv2
from time import time
xlnk = Xlnk()
overlay = Overlay('ORB_FPGA.bit')
img_list = []
for i in range (20):
    img_gray = cv2.imread('../test_data/'+str(i)+'.jpg',cv2.IMREAD_GRAYSCALE)
    src_buf = xlnk.cma_array(shape=(640*480,), dtype=np.uint8)
    view=np.frombuffer(src_buf,dtype = np.uint8,count = -1)
    np.copyto(view,img_gray.ravel(),casting='same_kind')
    img_list.append(src_buf)
des_buf = xlnk.cma_array(shape=(8192,10), dtype=np.int32)
dma_FAST = overlay.axi_dma_FAST
dma_Gaus = overlay.axi_dma_Gaus
dma_des = overlay.axi_dma_des_2Mem
dma_des.recvchannel.start()
dma_FAST.sendchannel.start()
dma_Gaus.sendchannel.start()

In [2]:
start = time()
for src_buf in img_list:
    dma_des.recvchannel.transfer(des_buf)
    dma_FAST.sendchannel.transfer(src_buf)
    dma_Gaus.sendchannel.transfer(src_buf)
    dma_Gaus.sendchannel.wait()
    dma_FAST.sendchannel.wait()
    dma_des.recvchannel.wait()
    des_buf.flush()
    #length = dma_des.mmio.read(0x58)
    #print(length/40)
stop = time()
print(str((stop-start)/20) + " 秒")
print(str(1/((stop-start)/20)) + " FPS")

0.017445957660675047 秒
57.3198685592423 FPS


In [7]:
src_buf.freebuffer()
des_buf.freebuffer()